## Install and import libraries

In [ ]:
!pip install datasets evaluate accelerate
!pip install mamba-ssm causal-conv1d>=1.2.0
# !pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)

In [ ]:
#https://colab.research.google.com/drive/1dUlEYnRbgJYg4_kofNpsCddLCh6vltNK?usp=sharing
!export LC_ALL="en_US.UTF-8"
!export LD_LIBRARY_PATH="/usr/lib64-nvidia"
!export LIBRARY_PATH="/usr/local/cuda/lib64/stubs"
!ldconfig /usr/lib64-nvidia

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbb.so.12 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_0.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link



In [ ]:
import os
import random
import json
import evaluate
import numpy as np

import torch
import torch.nn as nn
from torch.nn import BCEWithLogitsLoss

from functools import partial
from collections import namedtuple
from dataclasses import dataclass, field, asdict
from datasets import load_dataset
from transformers import Trainer, DataCollatorWithPadding, AutoTokenizer, TrainingArguments

from mamba_ssm.models.mixer_seq_simple import MixerModel, _init_weights # MambaLMHeadModel
from mamba_ssm.utils.generation import GenerationMixin
from mamba_ssm.utils.hf import load_config_hf, load_state_dict_hf

from huggingface_hub import HfApi

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

## Download Dataset

In [ ]:
imdb = load_dataset('imdb')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

## EDA

Dataset has 2 fields:
- text: movie review
- label: 1 - positive, 0 - negative

In [ ]:
imdb

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [ ]:
imdb['test'][0]

{'text': 'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as 

In [ ]:
imdb['unsupervised'][0]

{'text': 'This is just a precious little diamond. The play, the script are excellent. I cant compare this movie with anything else, maybe except the movie "Leon" wonderfully played by Jean Reno and Natalie Portman. But... What can I say about this one? This is the best movie Anne Parillaud has ever played in (See please "Frankie Starlight", she\'s speaking English there) to see what I mean. The story of young punk girl Nikita, taken into the depraved world of the secret government forces has been exceptionally over used by Americans. Never mind the "Point of no return" and especially the "La femme Nikita" TV series. They cannot compare the original believe me! Trash these videos. Buy this one, do not rent it, BUY it. BTW beware of the subtitles of the LA company which "translate" the US release. What a disgrace! If you cant understand French, get a dubbed version. But you\'ll regret later :)',
 'label': -1}

## Build Custom Mamba Model

In [ ]:
# Source: https://github.com/state-spaces/mamba/blob/main/mamba_ssm/models/config_mamba.py
@dataclass
class MambaConfig:
    d_model: int = 2_560
    d_intermediate: int = 0
    n_layer: int = 64 # ViT 12
    vocab_size: int = 50_277
    ssm_cfg: dict = field(default_factory=dict)
    rms_norm: bool = True
    residual_in_fp32: bool = True
    fused_add_norm: bool = True
    pad_vocab_size_multiple: int = 8

    def to_json_string(self):
        return json.dumps(asdict(self))

    def to_dict(self):
        return asdict(self)

In [ ]:
class MambaClassificationHead(nn.Module):
    def __init__(self, d_model, num_classes, **kwargs):
        super(MambaClassificationHead, self).__init__()
        # super().__init__()
        self.classification_head = nn.Linear(d_model, num_classes, **kwargs)

    def forward(self, hidden_states):
        return self.classification_head(hidden_states)

In [ ]:
class MambaLMHeadModel(nn.Module, GenerationMixin):
    def __init__(self,
                 config: MambaConfig,
                 initializer_cfg=None,
                 device=None,
                 dtype=None
                ) -> None:
        self.config = config
        d_model = config.d_model
        d_intermediate = config.d_intermediate
        n_layer = config.n_layer
        vocab_size = config.vocab_size
        ssm_cfg = config.ssm_cfg
        rms_norm = config.rms_norm
        residual_in_fp32 = config.residual_in_fp32
        fused_add_norm = config.fused_add_norm
        pad_vocab_size_multiple = config.pad_vocab_size_multiple
        factory_kwargs = {'device': device, 'dtype': dtype}

        super().__init__()
        if vocab_size % pad_vocab_size_multiple != 0:
            vocab_size += pad_vocab_size_multiple - (vocab_size % pad_vocab_size_multiple)

        self.backbone = MixerModel(
            d_model=d_model,
            d_intermediate=d_intermediate,
            n_layer=n_layer,
            vocab_size=vocab_size,
            ssm_cfg=ssm_cfg,
            rms_norm=rms_norm,
            initializer_cfg=initializer_cfg,
            residual_in_fp32=residual_in_fp32,
            fused_add_norm=fused_add_norm,
            **factory_kwargs,
        )
        self.lm_head = nn.Linear(d_model, vocab_size, bias=False, **factory_kwargs)

        self.apply(
            partial(
                _init_weights,
                n_layer=n_layer,
                **(initializer_cfg if initializer_cfg is not None else {}),
            )
        )

    def allocate_inference_cache(self, batch_size, max_seqlen, dtype=None, **kwargs):
        return self.backbone.allocate_inference_cache(batch_size, max_seqlen, dtype=dtype, **kwargs)

    def forward(self, input_ids, position_ids=None, inference_params=None, num_last_tokens=0):
        """
        "position_ids" is just to be compatible with Transformer generation. We don't use it.
        num_last_tokens: if > 0, only return the logits for the last n tokens
        """
        hidden_states = self.backbone(input_ids, inference_params=inference_params)
        if num_last_tokens > 0:
            hidden_states = hidden_states[:, -num_last_tokens:]
        lm_logits = self.lm_head(hidden_states)
        CausalLMOutput = namedtuple("CausalLMOutput", ["logits"])
        return CausalLMOutput(logits=lm_logits)

    @classmethod
    def from_pretrained(cls, pretrained_model_name, device=None, dtype=None, **kwargs):
        config_data = load_config_hf(pretrained_model_name)
        config = MambaConfig(**config_data)
        model = cls(config, device=device, dtype=dtype, **kwargs)
        model.load_state_dict(load_state_dict_hf(pretrained_model_name, device=device, dtype=dtype))
        return model

    def save_pretrained(self, save_directory):
        """
        Minimal implementation of save_pretrained for MambaLMHeadModel.
        Save the model and its configuration file to a directory.
        """
        # Ensure save_directory exists
        if not os.path.exists(save_directory):
            os.makedirs(save_directory)

        # Save the model's state_dict
        model_path = os.path.join(save_directory, 'pytorch_model.bin')
        torch.save(self.state_dict(), model_path)

        # Save the configuration of the model
        config_path = os.path.join(save_directory, 'config.json')
        with open(config_path, 'w') as f:
            json.dump(self.config.__dict__, f)

In [ ]:
class MambaTextClassification(MambaLMHeadModel):
    def __init__(self,
                 config: MambaConfig,
                 initializer_cfg=None,
                 device=None,
                 dtype=None,
                ) -> None:
        super().__init__(config, initializer_cfg, device, dtype)

        # Create a classification head tailored for Mamba models
        self.classification_head = MambaClassificationHead(d_model=config.d_model, num_classes=2)

        # Remove the default LM head (language modeling head) since we're focusing on text classification
        del self.lm_head

    def forward(self, input_ids, attention_mask=None, labels=None):
        # Pass inputs into base Mamba model to obtain hidden states
        hidden_states = self.backbone(input_ids) # (batch_size, seq_len, d_model)

        # Generate a representative feature by averaging hidden states across sequence length
        mean_hidden_states = hidden_states.mean(dim=1) # (batch_size, d_model)

        # Pass the mean hidden states into the classification head to obtain logits
        logits = self.classification_head(mean_hidden_states)

        # Handle training and evaluation scenarios
        if labels is None:
            # During evaluation, return only logits
            ClassificationOutput = namedtuple('ClassificationOutput', ['logits'])
            return ClassificationOutput(logits=logits)
        else:
            # During traning, calculate loss using CrossEntropyLoss
            ClassificationOutput = namedtuple('ClassificationOutput', ['loss', 'logits'])
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits, labels)

            return ClassificationOutput(loss=loss, logits=logits)

    def predict(self, text, tokenizer, id2label=None):
        # Tokenize input text
        input_ids = torch.tensor(tokenizer(text)['input_ids'], device='cuda')[None]
        with torch.no_grad():
            logits = self.forward(input_ids).logits[0]
            label = np.argmax(logits.cpu().numpy())

        # Return the label or the mapped label if id2label is provided
        if id2label is not None:
            return id2label[label]
        else:
            return label

    @classmethod
    def from_pretrained(cls, pretrained_model_name, device=None, dtype=None, **kwargs):
        # Load pretrained model configuration
        config_data = load_config_hf(pretrained_model_name)
        config = MambaConfig(**config_data)

        # Initialize model with configuration
        model = cls(config, device=device, dtype=dtype, **kwargs)

        # Load model state dictionary
        model_state_dict = load_state_dict_hf(pretrained_model_name, device=device, dtype=dtype)
        model.load_state_dict(model_state_dict, strict=False)

        print("Newly initialized embedding:", set(model.state_dict().keys()) - set(model_state_dict.keys()))
        return model

In [ ]:
# Load pretrained Mamba model
model = MambaTextClassification.from_pretrained('state-spaces/mamba-130m')
model.to('cuda')

tokenizer = AutoTokenizer.from_pretrained('EleutherAI/gpt-neox-20b')

tokenizer.pad_token_id = tokenizer.eos_token_id

pytorch_model.bin:   0%|          | 0.00/517M [00:00<?, ?B/s]

Newly initialized embedding: {'classification_head.classification_head.weight', 'classification_head.classification_head.bias'}


tokenizer_config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/457k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Data Preprocessing

In [ ]:
def preprocess(examples):
    samples = tokenizer(examples['text'], truncation=True)
    samples.pop('attention_mask')
    return samples

In [ ]:
tokenized_imdb = imdb.map(preprocess, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
random.seed(42)

train_dataset = tokenized_imdb['train']
test_dataset = tokenized_imdb['test']

# Since test dataset is large, 1% will be used to evaluate
total_samples = len(test_dataset)
eval_samples = int(0.2 * total_samples)
eval_indices = random.sample(range(total_samples), eval_samples)
eval_dataset = test_dataset.select(eval_indices)

## Evaluation

In [ ]:
acc = evaluate.load('accuracy')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    return acc.compute(predictions=predictions, references=labels)

## Training

In [ ]:
class MambaTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        input_ids = inputs.pop('input_ids')
        labels = inputs.pop('labels')

        outputs = model(input_ids=input_ids, labels=labels)

        loss = outputs.loss

        return (loss, outputs) if return_outputs else loss

    def save_model(self, output_dir=None, _internal_call=False):
        if output_dir is None:
            output_dir = self.args.output_dir

        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        torch.save(self.model.state_dict(), f'{output_dir}/pytorch_model.bin')

        self.tokenizer.save_pretrained(output_dir)

        with open(f'{output_dir}/config.json', 'w') as f:
            json.dump(self.model.config.to_dict(), f)

In [ ]:
os.environ['WANDB_PROJECT'] = 'mamba_text_cls'

training_args = TrainingArguments(
    output_dir='mamba_text_classification', # Folder to save train results
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    warmup_ratio=0.01,
    lr_scheduler_type='cosine',
    report_to='none',
    evaluation_strategy='steps',
    eval_steps=0.1,
    save_strategy='steps',
    save_steps=0.1,
    logging_strategy='steps',
    logging_steps=0.1,
    # push_to_hub=True,
    load_best_model_at_end=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = MambaTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy
625,0.445700,0.336933,0.885400
1250,0.376100,0.241587,0.924600
1875,0.287200,0.306217,0.923000
2500,0.318000,0.276116,0.919800
3125,0.284500,0.226148,0.939800
3750,0.282900,0.235643,0.941000
4375,0.243100,0.235099,0.941600
5000,0.240700,0.230278,0.941800
5625,0.217400,0.225707,0.942600
6250,0.245400,0.224520,0.942600


Step,Training Loss,Validation Loss,Accuracy
625,0.445700,0.336933,0.885400
1250,0.376100,0.241587,0.924600
1875,0.287200,0.306217,0.923000
2500,0.318000,0.276116,0.919800
3125,0.284500,0.226148,0.939800
3750,0.282900,0.235643,0.941000
4375,0.243100,0.235099,0.941600
5000,0.240700,0.230278,0.941800
5625,0.217400,0.225707,0.942600
6250,0.245400,0.224520,0.942600


TrainOutput(global_step=6250, training_loss=0.29409537109375, metrics={'train_runtime': 6701.7381, 'train_samples_per_second': 3.73, 'train_steps_per_second': 0.933, 'total_flos': 0.0, 'train_loss': 0.29409537109375, 'epoch': 1.0})

In [ ]:
# Push model to hugging face
trainer.push_to_hub(commit_message='Training Complete')

pytorch_model.bin:   0%|          | 0.00/517M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/toan-ly/mamba_text_classification/commit/82cef4e1696b24321fb2166eb6db7aff684d9293', commit_message='Training Complete', commit_description='', oid='82cef4e1696b24321fb2166eb6db7aff684d9293', pr_url=None, pr_revision=None, pr_num=None)

## Testing

In [ ]:
outputs = trainer.predict(test_dataset)

In [ ]:
print(outputs.metrics)

{'test_loss': 0.21847689151763916, 'test_accuracy': 0.94344, 'test_runtime': 1427.3513, 'test_samples_per_second': 17.515, 'test_steps_per_second': 1.095}


## Inference

In [ ]:
model = MambaTextClassification.from_pretrained('toan-ly/mamba_text_classification')
model.to('cuda')

tokenizer = AutoTokenizer.from_pretrained('toan-ly/mamba_text_classification')
tokenizer.pad_token_id = tokenizer.eos_token_id

Newly initialized embedding: set()


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}

In [ ]:
text = imdb['test'][0]['text']
label = imdb['test'][0]['label']

In [ ]:
response = model.predict(text, tokenizer, id2label)

In [ ]:
print(f'Classify: {text}\nGround Truth: {id2label[label]}\nPredict: {response}')

Classify: I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn't match the background, and painfully one-dimensional characters cannot be overcome with a 'sci-fi' setting. (I'm sure there are those of you out there who think Babylon 5 is good sci-fi TV. It's not. It's clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It's really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it's rubbish as they hav

### Source: AI VIETNAM